In [2]:
import numpy as np
import pandas as pd
from hazm import stopwords_list, Normalizer, WordTokenizer, SentenceTokenizer, Stemmer, Lemmatizer, sent_tokenize, word_tokenize
import docx
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import requests
from sklearn.metrics.pairwise import cosine_similarity
from rake_nltk import Rake

In [3]:
class PersianRAKE(Rake):
    def _tokenize_text_to_sentences(self, text: str):
        return sent_tokenize(text)

    def _tokenize_sentence_to_words(self, sentence: str):
        return word_tokenize(sentence)

In [4]:
def read_from_docx(doc):
    fullText=''
    for pra in doc.paragraphs:
        fullText+=pra.text+' '

    return fullText

def split_into_overlapping_chunks(sentences, max_chunk_size=1000, overlap_size=200):
    chunks = []
    current_chunk = ""
    current_chunk_size = 0

    for sentence in sentences:
        sentence_length = len(sentence)

        if current_chunk_size + sentence_length > max_chunk_size and current_chunk:
            chunks.append(current_chunk.strip())

            overlap_buffer = current_chunk[-overlap_size:].strip() if current_chunk else ""
            current_chunk = overlap_buffer + " "
            current_chunk_size = len(overlap_buffer) + 1

        current_chunk += sentence + " "
        current_chunk_size += sentence_length + 1

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

def preprocess_text_1(text):
    # text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'( +)', ' ', str(text))
    return text


def preprocess_text_2(text):
    text = re.sub('(\(.*?\))|(\[.*?\])', '', str(text))
    text = re.sub(r'( +)', ' ', str(text))

    word_tokenizer = WordTokenizer()
    words = word_tokenizer.tokenize(text)

    stopwords = stopwords_list()
    filtered_words = [word for word in words if word not in stopwords]

    lemmatizer = Lemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    return ' '.join(lemmatized_words)


def check_spelling(main_text):
    endpoint = "https://api.languagetool.org/v2/check"

    data = {
        "text": main_text,
        "language": "en-US",
    }

    response = requests.post(endpoint, data=data)
    json_response = response.json()

    updated_text = main_text

    for match in json_response.get("matches", []):
        replacement = match["replacements"][0]["value"] if match["replacements"] else ""

        offset = match["offset"]
        length = match["length"]

        updated_text = updated_text.replace(main_text[offset:offset+length], replacement)

    print("Original query: ", main_text)
    print("Corrected query: ", updated_text)
    return updated_text


def extract_key_phrases(query, max_phrases, lang="persian"):
    persian_stopwords = stopwords_list()
    r = PersianRAKE(
        stopwords=persian_stopwords,
        max_length= 5
    )

    r.extract_keywords_from_text(query)
    phrases = r.get_ranked_phrases()

    print("Key phrases:", phrases[:2])

    multi_word_phrases = [p for p in phrases if len(p.split()) > 1]

    return multi_word_phrases[:max_phrases]

def phrase_search(query, documents):
    phrase = query.strip()
    matched_docs = []

    for idx, doc in documents.iterrows():
        if phrase in doc['preprocessed_text']:
            matched_docs.append(idx)

    return matched_docs

In [5]:
# Loading the main document
document = read_from_docx(docx.Document("nutrients.docx"))

In [6]:
# Chunking The Document
normalizer = Normalizer()
normalized_text = normalizer.normalize(document)

sentence_tokenizer = SentenceTokenizer()
sentences = sentence_tokenizer.tokenize(normalized_text)

max_chunk_size = 1000
overlap_size = 200
chunks = split_into_overlapping_chunks(sentences, max_chunk_size, overlap_size)
print("<Chunk 1>")
print(f"Original Chunk:\n{chunks[0]}")

# Preprocessing The Chunks
preprocessed1_chunks = [preprocess_text_1(chunk) for chunk in chunks]

preprocessed2_chunks = [preprocess_text_2(chunk) for chunk in preprocessed1_chunks]
print(f"Preprocessed Chunk:\n{chunks[0]}")

<Chunk 1>
Original Chunk:
تغذیه علمی است که در آن اثر متقابل مواد مغذی و هر آنچه در غذا یافت می‌شود را در رابطه با حفظ، رشد، تولید و سلامت ارگان‌های بدن مورد تفسیر قرار می‌دهد. این علم شامل بررسی دریافت مواد غذایی، جذب، ادغام، تولید ترکیبات توسط موجودات زنده، دگرگونی و ترشح می‌شود. رژیم غذایی یک ارگانیسم متشکل است از هر آنچه که دریافت می‌کند و به طور گسترده‌ای تحت تاثیر میزان دسترسی و دلپذیری آن ماده غذایی قرار دارد. برای انسان‌ها یک رژیم سالم شامل مراحل تهیه و روش‌های نگهداری غذا جهت جلوگیری از اکسیداسیون، حرارت و از بین رفتن مواد مغذی و همچنین به حداقل رساندن مسمومیت غذایی می‌باشد. یک تغذیه ناسالم باعث ایجاد بیماری‌های مختلف می‌شود. کمبود برخی مواد مغذی به بیماری‌های مانند کوری، کم خونی، اسکوربوت، تولد نوزاد نارس، مرگ جنین، کرتنیسم یا دریافت مازاد برخی مواد مغذی به بیماری‌هایی چون چاقی، سندروم متابولیک، بیماری‌های قلبی و عروقی، دیابت و استئوپروز منجر می‌شوند.
Preprocessed Chunk:
تغذیه علمی است که در آن اثر متقابل مواد مغذی و هر آنچه در غذا یافت می‌شود را در رابطه با حفظ، رشد، تولید و

In [7]:
query = "چند درصد از بدن پروتئین است؟"
query = check_spelling(query)
target_phrases = extract_key_phrases(query, max_phrases=2)

processed_query = preprocess_text_2(preprocess_text_1(query))

Original query:  چند درصد از بدن پروتئین است؟
Corrected query:  چند درصد از بدن پروتئین است؟
Key phrases: ['بدن پروتئین', 'درصد']


In [8]:
vectorizer = TfidfVectorizer(ngram_range=(1, 3))
tfidf_matrix = vectorizer.fit_transform(preprocessed2_chunks)
query_vector = vectorizer.transform([processed_query])

cosine_similarities = cosine_similarity(query_vector, tfidf_matrix)[0]

phrase_match_bonus = np.zeros(len(preprocessed1_chunks))
for i, chunk in enumerate(preprocessed1_chunks):
    for phrase in target_phrases:
        if phrase in chunk:
            phrase_match_bonus[i] += 1.0

hybrid_scores = cosine_similarities + (phrase_match_bonus * 0.5)

top_k = 3
indices = np.argsort(-hybrid_scores)[:top_k]

top_3 = [preprocessed1_chunks[idx] for idx in indices]

print("Top 3 Hybrid Results:")
for idx, chunk in zip(indices, top_3):
    print(f"Chunk {idx + 1} (Score: {hybrid_scores[idx]:.2f}):\n{chunk}")

Top 3 Hybrid Results:
Chunk 34 (Score: 0.22):
چربیها اشاره کنیم که انرژی بدن را تامین میکنند ریزمغذیها نیز به موادی مربوط میشوند که مصرف مقادیر کمی از آنها برای بدن کافی است برای مثال میتوانیم انواع ویتامینها و مواد معدنی را معرفی کنیم در نتیجه ۶ گروه اصلی از درشتمغذیها و ریزمغذیها وجود دارند که برای بدن بسیار ضروری هستند ۱ پروتئین از مهمترین مواد مغذی بدن مصرف پروتئین اهمیت ویژهای دارد و این اهمیت تنها محدود به ورزشکاران نمیشود بلکه پروتئین برای داشتن سلامت بدن ضروری بوده و تمامی افراد باید آن را مصرف کنند در حقیقت پروتئین خشتهای ساختمان بدن است و نقش آن فقط به عضله سازی محدود نمیشود هر سلولی از استخوان تا پوست و مو حاوی پروتئین بوده و برای عملکرد سالم نیاز به مواد دارای پروتئین دارند جالب است بدانید ۱۶ درصد از کل وزن بدن یک فرد معمولی را پروتئین تشکیل میدهد و این درصدی حیرتآور است اساسا پروتئین برای رشد سلامتی و حفظ بدن استفاده میشود ناگفته نماند هورمونهای بدن پادتنها و دیگر مواد مهم بدن از جنس پروتئین هستند
Chunk 42 (Score: 0.12):
ات اخیر نشان داده است که چربیهای سالم جزئی مهم از یک

In [9]:
api_keys = pd.DataFrame()
api_keys = pd.read_csv('api_key.csv')

In [12]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=api_keys.loc[0, 'api_key'],
)

completion = client.chat.completions.create(
  extra_headers={
  },
  model="deepseek/deepseek-r1",
  messages=[
    {
      "role": "user",
      "content": f"سه تا داکیومنت دارم:\n\n1. {top_3[0]}\n\n2. {top_3[1]}\n\n3. {top_3[2]}\n\nسوال: {query}\n\nبه طور خلاصه(در حد یک پاراگراف) با توجه به داکیومنت ها جواب سوال رو بده و اشاره ای به کلمه داکیومنت نکن"
    }
  ]
)
print(completion.choices[0].message.content)

پروتئین یکی از مواد مغذی اساسی است که حدود ۱۶ درصد از وزن بدن یک فرد معمولی را تشکیل میدهد. این ماده نهتنها در ساختار عضلات، بلکه در تمام سلولها از جمله استخوان، پوست و مو نقش حیاتی ایفا میکند. پروتئینها همچنین اجزای اصلی هورمونها، پادتنها و سایر ترکیبات ضروری بدن هستند. هرچند بدن در شرایط عادی از پروتئین بهعنوان منبع اولیه انرژی استفاده نمیکند، اما در مواقع ضروری میتواند به آن متکی باشد. برای حفظ عملکرد سالم، نیاز به دریافت آمینواسیدهای ضروری از طریق تغذیه وجود دارد که در منابع غذایی مختلف یافت میشوند و بدن قادر است با ترکیب آنها، پروتئینهای کامل موردنیاز خود را تولید کند.
